导入

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

设置超参数

In [2]:
train_epochs = 100
batch_size = 10
learning_rate = 0.001
display_step = 100

导入数据

In [3]:
df = pd.read_csv('data/boston.csv')
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,37.970000,50.000000


格式化数据

In [4]:
data = np.array(df.values)
x_data = data[:, :12]
y_data = data[:, 12].reshape([-1, 1])
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)) \
    .shuffle(1000).repeat(train_epochs).batch(batch_size)

iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

Instructions for updating:
Colocations handled automatically by placer.


构建模型

In [5]:
x = tf.placeholder(tf.float32, [None, 12], name='x')
y = tf.placeholder(tf.float32, [None, 1], name='y')

w = tf.Variable(tf.truncated_normal([12, 1], stddev=0.01), name='w')
b = tf.Variable(0.0, name='b')
pred = tf.matmul(x, w) + b

定义损失函数

In [6]:
loss_function = tf.reduce_mean(tf.pow(y-pred,2))

定义优化器

In [7]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

初始化

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(iterator.initializer)

开始训练

In [9]:
step = 0
while 1:
    try:
        xs, ys = sess.run(next_element)
    except tf.errors.OutOfRangeError:
        break

    sess.run([optimizer], feed_dict={x: xs, y: ys})
    step += 1
    if step % display_step == 0:
        loss = sess.run(loss_function, feed_dict={x: xs, y: ys})
        print('step:{:d} loss:{:.05f}'.format(step, loss))


step:100 loss:134.35596
step:200 loss:51.48635
step:300 loss:75.70670
step:400 loss:33.47987
step:500 loss:179.32510
step:600 loss:56.63580
step:700 loss:45.38510
step:800 loss:21.74374
step:900 loss:57.96425
step:1000 loss:131.43283
step:1100 loss:37.82201
step:1200 loss:80.25568
step:1300 loss:146.63162
step:1400 loss:96.89964
step:1500 loss:51.99840
step:1600 loss:68.34146
step:1700 loss:119.83633
step:1800 loss:64.79706
step:1900 loss:10.57024
step:2000 loss:17.23127
step:2100 loss:53.32644
step:2200 loss:18.51587
step:2300 loss:31.06733
step:2400 loss:95.26332
step:2500 loss:31.82914
step:2600 loss:47.57215
step:2700 loss:60.82949
step:2800 loss:31.45026
step:2900 loss:47.79180
step:3000 loss:104.56612
step:3100 loss:62.96096
step:3200 loss:9.39615
step:3300 loss:48.80781
step:3400 loss:23.03950
step:3500 loss:47.11185
step:3600 loss:48.29556
step:3700 loss:41.03374
step:3800 loss:75.40704
step:3900 loss:92.37180
step:4000 loss:51.53376
step:4100 loss:23.38017
step:4200 loss:58.12

显示w,b

In [10]:
print('w:{}\nb:{}'.format(sess.run(w),sess.run(b)))

w:[[-1.0553875e-01]
 [ 9.1842026e-02]
 [-8.4071495e-02]
 [ 1.6937598e+00]
 [ 8.8273799e-01]
 [ 1.4681091e+00]
 [ 1.2169045e-01]
 [ 4.1252479e-01]
 [-8.6856917e-02]
 [ 1.6383628e-03]
 [ 6.4463890e-01]
 [-8.2623118e-01]]
b:1.148775339126587
